In [1]:
import nibabel as nib

In [2]:
import numpy as np

In [3]:
import networkx as nx

In [4]:
import os

In [5]:
import math

In [6]:
import heapq

In [24]:
import pandas as pd

In [7]:
def get_strength(adjacency_matrix):
    strength = np.sum(adjacency_matrix,axis = 1)
    strength = strength[:,np.newaxis]
   
    return strength

In [9]:
def get_inverse_participation_ratio(adjacency_matrix,strength):
    ratio_matrix = np.divide(adjacency_matrix,strength,out = np.zeros_like(adjacency_matrix),where=strength != 0)
    pow_matrix= np.power(ratio_matrix,2)
    inverse_participation_ratio = np.sum(pow_matrix,axis = 1)
    inverse_participation_ratio = inverse_participation_ratio[:,np.newaxis]
    
    return inverse_participation_ratio

In [10]:
def get_graph_details(graph_degree_view):
    deg_list = [val for (node,val) in graph_degree_view]
    deg_array = np.array(deg_list)
    
    max_index = heapq.nlargest(1, range(len(deg_array)), key=deg_array.__getitem__)
    dmax = deg_array[max_index[0]]
    
    min_index = heapq.nsmallest(1, range(len(deg_array)), key=deg_array.__getitem__)
    dmin = deg_array[min_index[0]]
    
    count_degree_nodes = {}
    for i in range(dmin,dmax+1):
        deg_count = deg_list.count(i)
    
        if deg_count != 0:
            count_degree_nodes[i] = deg_count
            
    return deg_list,deg_array,count_degree_nodes,dmax,dmin

In [54]:
def get_strength_means(graph_degree_array,count_degree_nodes,degree_list,strength):
    strength_means = {}
    for key in count_degree_nodes.keys():
        key_array = np.linspace(key,key,num = len(degree_list))
    
        #kronecked delta function
        delta = np.equal(graph_degree_array,key_array)
        delta = delta[:,np.newaxis]
        
        #calculate conditional means of strength
        KD_result = np.multiply(strength,delta)
        sum_result = np.sum(KD_result)
        strength_means[key] = (1/count_degree_nodes[key]) * sum_result

    return strength_means

In [55]:
def get_inverse_participation_ratio_means(graph_degree_array,count_degree_nodes,
                                          degree_list,inverse_participation_ratio):
    
    inv_participation_means = {}
    for key in count_degree_nodes.keys():
        key_array = np.linspace(key,key,num = len(degree_list))
        
        #kronecked delta function
        delta = np.equal(graph_degree_array,key_array)
        delta = delta[:,np.newaxis]
        
        #calculate conditional means of inverse participation ratio
        INV_result = np.multiply(inverse_participation_ratio,delta)
        sum_result = np.sum(INV_result)
        inv_participation_means[key] = (1/count_degree_nodes[key]) * sum_result
    
    return inv_participation_means

In [87]:
def get_graph_features(G):
    adj_matrix = nx.to_numpy_array(G) 
    strength = get_strength(adj_matrix)
    inv_participation_ratio = get_inverse_participation_ratio(adj_matrix,strength)
    deg_list,deg_array,count_degree_nodes,dmax,dmin = get_graph_details(G.degree)
    strength_means = get_strength_means(deg_array,count_degree_nodes,deg_list,strength)
    inv_participation_means = get_inverse_participation_ratio_means(deg_array,count_degree_nodes,
                                                                deg_list,inv_participation_ratio)
    strength_df = pd.DataFrame(strength)
    inv_part_ratio_df = pd.DataFrame(inv_participation_ratio)
    node_df = pd.concat([strength_df,inv_part_ratio_df],axis=1)
    node_df.columns = ['strength','inverse_participation_ratio']
    
    #get strength_means 1D array
    str_means = np.zeros([840,1])   
    for idx in range(0,840):
        try:
            str_means[idx] = strength_means[idx]
        except:
            continue
            
    #get inv participation means 1D array
    inv_part_means = np.zeros([840,1])
    for idx in range(0,840):
        try:
            inv_part_means[idx] = inv_participation_means[idx]
        except:
            continue
    
    strength_means_df = pd.DataFrame(str_means)       
    inv_part_means_df = pd.DataFrame(inv_part_means)
    degree_df = pd.concat([strength_means_df,inv_part_means_df],axis = 1)
    degree_df.columns = ['strength_means','inverse_participation_ratio_means']
    
    return node_df,degree_df

In [13]:
G = nx.read_gml(r'D:\images\smwc images\CN vs AD\GM\Thresholded_graphs\Training\graph_threshold_0.3\smwc11.gml')

In [14]:
adj_matrix = nx.to_numpy_array(G)

In [15]:
strength = get_strength(adj_matrix)

In [16]:
inv_participation_ratio = get_inverse_participation_ratio(adj_matrix,strength)

In [17]:
deg_list,deg_array,count_degree_nodes,dmax,dmin = get_graph_details(G.degree)

In [18]:
strength_means = get_strength_means(deg_array,count_degree_nodes,deg_list,strength)

In [19]:
inv_participation_means = get_inverse_participation_ratio_means(deg_array,count_degree_nodes,
                                                                deg_list,inv_participation_ratio)

In [39]:
strength_df = pd.DataFrame(strength)

In [40]:
inv_part_ratio_df = pd.DataFrame(inv_participation_ratio)

In [41]:
node_df = pd.concat([strength_df,inv_part_ratio_df],axis=1)

In [43]:
node_df.columns = ['strength','inverse_participation_ratio']

In [51]:
len(strength_means)

172

In [76]:
str_means = np.zeros([840,1])

#get strength_means 1D array
for idx in range(0,840):
    try:
        str_means[idx] = strength_means[idx]
    except:
        continue

In [77]:
#get inv participation means 1D array
inv_part_means = np.zeros([840,1])
for idx in range(0,840):
    try:
        inv_part_means[idx] = inv_participation_means[idx]
    except:
        continue

In [78]:
strength_means_df = pd.DataFrame(str_means)

In [79]:
inv_part_means_df = pd.DataFrame(inv_part_means)

In [80]:
degree_df = pd.concat([strength_means_df,inv_part_means_df],axis = 1)

In [83]:
degree_df.columns = ['strength_means','inverse_participation_ratio_means']

# CN vs AD

Training

In [96]:
graph_path = 'D:\\images\\smwc images\\CN vs AD\\GM\\Thresholded_graphs\\Training\\graph_threshold_0.3\\'
graph_list = os.listdir(graph_path)
write_path = 'D:\\images\\smwc images\\CN vs AD\\GM\\Thresholded_graphs\\Features\\Training\\graph_threshold_0.3\\'

for graph in graph_list:
    if 'multiplex' not in graph:
        G = nx.read_gml(graph_path+graph)
        node_df,degree_df = get_graph_features(G)
        file_name = graph[0:len(graph) - 4]
        node_df.to_csv(write_path+file_name+"_node.csv",index=False)
        degree_df.to_csv(write_path+file_name+"_degree.csv",index=False)
        print(file_name)
        print("**************************************************")

smwc11
**************************************************
smwc110
**************************************************
smwc111
**************************************************
smwc112
**************************************************
smwc113
**************************************************
smwc114
**************************************************
smwc115
**************************************************
smwc116
**************************************************
smwc117
**************************************************
smwc118
**************************************************
smwc119
**************************************************
smwc12
**************************************************
smwc120
**************************************************
smwc121
**************************************************
smwc122
**************************************************
smwc123
**************************************************
smwc124
**************************************************

Testing

In [97]:
graph_path = 'D:\\images\\smwc images\\CN vs AD\\GM\\Thresholded_graphs\\Testing\\graph_threshold_0.3\\'
graph_list = os.listdir(graph_path)
write_path = 'D:\\images\\smwc images\\CN vs AD\\GM\\Thresholded_graphs\\Features\\Testing\\graph_threshold_0.3\\'

for graph in graph_list:
    if 'multiplex' not in graph:
        G = nx.read_gml(graph_path+graph)
        node_df,degree_df = get_graph_features(G)
        file_name = graph[0:len(graph) - 4]
        node_df.to_csv(write_path+file_name+"_node.csv",index=False)
        degree_df.to_csv(write_path+file_name+"_degree.csv",index=False)
        print(file_name)
        print("**************************************************")

smwc1100
**************************************************
smwc141
**************************************************
smwc142
**************************************************
smwc143
**************************************************
smwc144
**************************************************
smwc145
**************************************************
smwc146
**************************************************
smwc147
**************************************************
smwc148
**************************************************
smwc149
**************************************************
smwc150
**************************************************
smwc191
**************************************************
smwc192
**************************************************
smwc193
**************************************************
smwc194
**************************************************
smwc195
**************************************************
smwc196
***********************************************